# Clustering Crypto

In [78]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [79]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [80]:
# Create a DataFrame 

In [81]:
# Alternatively, use the provided csv file:
csv = pd.read_csv("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.DataFrame(csv)
print(list(crypto_df.columns))

['Unnamed: 0', 'CoinName', 'Algorithm', 'IsTrading', 'ProofType', 'TotalCoinsMined', 'TotalCoinSupply']


### Data Preprocessing

In [82]:
# Keep only necessary columns:
columns = ['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']
crypto_df.drop(columns=[column for column in crypto_df.columns if column not in columns], inplace=True)
crypto_df.head()


,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [83]:
# Keep only cryptocurrencies that are trading
# Citation: https://stackoverflow.com/questions/18172851/deleting-dataframe-row-in-pandas-based-on-column-value
crypto_df = crypto_df[crypto_df.IsTrading != False]
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,Super Zero,Ethash,True,PoW,NaN,1000000000
1244,UOS,SHA-256,True,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,True,PoW,7.296538e+06,21000000


In [84]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df[crypto_df.Algorithm != False]
set(crypto_df.Algorithm)

{'1GB AES Pattern Search',
 '536',
 'Argon2',
 'Argon2d',
 'BLAKE256',
 'Blake',
 'Blake2S',
 'Blake2b',
 'C11',
 'Cloverhash',
 'Counterparty',
 'CryptoNight',
 'CryptoNight Heavy',
 'CryptoNight Heavy X',
 'CryptoNight-Lite',
 'CryptoNight-V7',
 'CryptoNight-lite',
 'Cryptonight-GPU',
 'Curve25519',
 'DPoS',
 'Dagger',
 'Dagger-Hashimoto',
 'ECC 256K1',
 'Equihash',
 'Equihash+Scrypt',
 'Equihash1927',
 'Ethash',
 'Exosis',
 'Green Protocol',
 'Groestl',
 'HMQ1725',
 'HybridScryptHash256',
 'IMesh',
 'Jump Consistent Hash',
 'Keccak',
 'Leased POS',
 'Lyra2RE',
 'Lyra2REv2',
 'Lyra2Z',
 'M7 POW',
 'Momentum',
 'Multiple',
 'NIST5',
 'NeoScrypt',
 'Ouroboros',
 'PHI1612',
 'POS 2.0',
 'POS 3.0',
 'PoS',
 'Progressive-n',
 'Proof-of-Authority',
 'Proof-of-BibleHash',
 'QUAIT',
 'QuBit',
 'Quark',
 'QuarkTX',
 'Rainforest',
 'SHA-256',
 'SHA-256 + Hive',
 'SHA-256D',
 'SHA-512',
 'SHA3',
 'SHA3-256',
 'Scrypt',
 'Scrypt-n',
 'Semux BFT consensus',
 'Shabal256',
 'Skein',
 'SkunkHash',
 

In [85]:
# Remove the "IsTrading" column
crypto_df.drop(columns='IsTrading', inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365Coin,X11,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,PoW,NaN,611000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
1243,Super Zero,Ethash,PoW,NaN,1000000000
1244,UOS,SHA-256,DPoI,NaN,1000000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [86]:
# Remove rows with at least 1 null value
crypto_df = crypto_df[crypto_df.TotalCoinsMined.isna()==False]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,SHA-256,PoW/PoS,0.000000e+00,0
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [87]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df.dropna(inplace=True)
crypto_df = crypto_df[crypto_df.TotalCoinsMined > 0]
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
1238,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,Horizen,Equihash,PoW,7.296538e+06,21000000


In [88]:
# Drop rows where there are 'N/A' text values
for column in crypto_df.columns:
    crypto_df = crypto_df[crypto_df.notna()]

In [89]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
names_list = [name for name in crypto_df.CoinName]
#names_set = set(names_list)
coins_name = pd.DataFrame(names_list)
coins_name.index = crypto_df.index

In [90]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df.drop(columns='CoinName', inplace=True)

In [91]:
# Create dummy variables for text features
X = pd.get_dummies(crypto_df,columns=['Algorithm','ProofType'])

In [93]:
# Standardize data
data_scaler = StandardScaler()
crypto_scaled = data_scaler.fit_transform(X)
crypto_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

### Reducing Dimensions Using PCA

In [103]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)

array([[-0.33732654,  1.03396523, -0.54017227],
       [-0.32069838,  1.03422943, -0.54053712],
       [ 2.2969926 ,  1.73293611, -0.62408996],
       ...,
       [ 0.31392264, -2.32212102,  0.3533829 ],
       [-0.14575953, -1.90216341,  0.34451283],
       [-0.30298085,  0.83165074, -0.22779592]])

In [95]:
# Create a DataFrame with the principal components data
df_crypto_pca = pd.DataFrame(data=crypto_pca, columns=["PC1", "PC2", "PC3"], index = crypto_df.index)
df_crypto_pca.head()

,PC1,PC2,PC3
0,-0.347304,1.035550,-0.535221
2,-0.330644,1.035842,-0.535559
5,2.299304,1.546692,-0.606049
7,-0.136665,-1.371799,0.195147
8,-0.154723,-2.057136,0.384285


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [96]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [97]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)
# Fit the model
model.fit(df_crypto_pca)
# Predict clusters
predictions = model.predict(df_crypto_pca)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
df_crypto_pca["Class"] = model.labels_
clustered_df = pd.concat([crypto_df, df_crypto_pca], axis="columns")
clustered_df['CoinName'] = names_list
clustered_df.index = crypto_df.index
clustered_df.Class.value_counts()

0    287
1    239
3      5
2      1
Name: Class, dtype: int64

### Visualizing Results

#### 3D-Scatter with Clusters

In [98]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC3",
    y="PC2",
    z="PC1",
    color="Class",
    symbol="Class",
    width=800,
    hover_name="CoinName",
    hover_data=["Algorithm"]
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


#### Table of Tradable Cryptocurrencies

In [99]:
# Table with tradable cryptos
tbl = clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "Class"])
tbl

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [100]:
# Print the total number of tradable cryptocurrencies
print(len(clustered_df.CoinName))

532


#### Scatter Plot with Tradable Cryptocurrencies

In [129]:
# Scale data to create the scatter plot
min_max_scaler = MinMaxScaler()
X = clustered_df[["TotalCoinsMined","TotalCoinSupply"]]

scaler = min_max_scaler.fit(X)
scaled = scaler.transform(X)

clustered_df[["TotalCoinsMined","TotalCoinSupply"]] = scaled
clustered_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,Class,CoinName
0,Scrypt,PoW/PoS,0.000000e+00,4.200000e-11,-0.347304,1.035550,-0.535221,0,42 Coin
2,Scrypt,PoW/PoS,1.065855e-03,5.320000e-04,-0.330644,1.035842,-0.535559,0,404Coin
5,X13,PoW/PoS,2.957551e-02,3.141593e-01,2.299304,1.546692,-0.606049,0,EliteCoin
7,SHA-256,PoW,1.810842e-05,2.100000e-05,-0.136665,-1.371799,0.195147,1,Bitcoin
8,Ethash,PoW,1.087731e-04,0.000000e+00,-0.154723,-2.057136,0.384285,1,Ethereum
...,...,...,...,...,...,...,...,...,...
1238,SHA-256,DPoS,2.020225e-03,2.000000e-03,2.480604,0.624471,0.124019,0,ZEPHYR
1242,Scrypt,PoW/PoS,1.508199e-05,2.500000e-04,-0.345350,1.035450,-0.535247,0,Gapcoin
1245,CryptoNight,PoW,9.901351e-04,1.400223e-03,0.326748,-2.323523,0.414429,1,Beldex
1246,Equihash,PoW,7.370282e-06,2.100000e-05,-0.145351,-1.887091,0.362300,1,Horizen


In [130]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply", hover_cols=["CoinName"])

:Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)